In [1]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
import optuna

In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v02.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [1149, 4836, 9443, 7483, 3492, 6287,  830, 3294, 5932,  815, 7070,
       1529, 3454, 6073, 7831, 1069, 3632,  953,  368, 3999, 9981,  906,
       6792, 1439, 8439, 9240, 8162,   32, 5785, 1013, 2747, 2702, 4807,
       831, 3985, 4003, 4758, 7955, 3976, 5436, 8634, 2511, 8627, 6888,
       4179, 1565, 9181, 6389, 5879, 1216, 8749, 8996,   99, 1481, 4120,
       8561, 7008, 5624,  271, 5204, 9169, 2537, 4070, 5458, 7383,  137,
       8414, 6327, 3119, 5659, 2907, 9390, 7055, 9770, 9614, 6769,  694,
       7854, 9925, 7895, 5259, 9598, 6352, 7884, 5592, 7017, 7374, 2208,
       7364, 1995, 7264, 2520, 3922, 7704, 2333, 1601, 2534, 6556, 9072,
       3046]

In [3]:
data = pl.scan_parquet(dataset_path + dataset_file).filter(pl.col("foto_mes") == 202108).collect()

In [4]:
# convirto a pandas dataframe
data = data.to_pandas()

In [5]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [6]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [7]:
X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)

### Predicciones para Kaggle

In [8]:
# Voy a probar 20 puntos de corte para cada semilla (total 100 envios)

version = 'v015' # UPADTE

modelos = [f'lgb_competencia2_{version}_s{semilla}_final.txt' for semilla in semillas]

In [9]:
len(modelos)

100

In [10]:
modelos_path

'C:/Eugenio/Maestria/DMEyF/modelos/'

In [10]:
X_test_with_predictions = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()

In [11]:
counter = 0

for modelo in modelos:
    
    # cargamos el modelo
    model = lgb.Booster(model_file=f'{modelos_path}{version}/{modelo}')
    
    # predecimos para agosto
    X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()
    y_pred_lgm = model.predict(X_test)
    X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm
    
    print(f'Predicciones del modelo: {modelo} DONE')
    
    counter += 1

Predicciones del modelo: lgb_competencia2_v015_s1149_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s4836_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s9443_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s7483_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s3492_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s6287_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s830_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s3294_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s5932_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s815_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s7070_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s1529_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s3454_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s6073_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s78

C:\Users\eaneg\AppData\Local\Temp\ipykernel_2500\3572328474.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm


Predicciones del modelo: lgb_competencia2_v015_s9072_final.txt DONE
Predicciones del modelo: lgb_competencia2_v015_s3046_final.txt DONE


C:\Users\eaneg\AppData\Local\Temp\ipykernel_2500\3572328474.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm


In [12]:
X_test_with_predictions = X_test_with_predictions.copy()

In [13]:
X_test_with_predictions['predicted_avg'] = X_test_with_predictions.iloc[:, -100:].mean(axis=1)

In [14]:
# ordeno de mayor probabilidad de baja a menor
idx = np.argsort(X_test_with_predictions['predicted_avg'])[::-1]
X_test_with_predictions.reset_index(drop=True, inplace=True)
X_test_with_predictions = X_test_with_predictions.iloc[idx]

In [20]:
# aux = X_test_with_predictions.head(100)
# aux

In [15]:
puntos_corte = np.linspace(9000, 13000, 100).astype(int)
puntos_corte

array([ 9000,  9040,  9080,  9121,  9161,  9202,  9242,  9282,  9323,
        9363,  9404,  9444,  9484,  9525,  9565,  9606,  9646,  9686,
        9727,  9767,  9808,  9848,  9888,  9929,  9969, 10010, 10050,
       10090, 10131, 10171, 10212, 10252, 10292, 10333, 10373, 10414,
       10454, 10494, 10535, 10575, 10616, 10656, 10696, 10737, 10777,
       10818, 10858, 10898, 10939, 10979, 11020, 11060, 11101, 11141,
       11181, 11222, 11262, 11303, 11343, 11383, 11424, 11464, 11505,
       11545, 11585, 11626, 11666, 11707, 11747, 11787, 11828, 11868,
       11909, 11949, 11989, 12030, 12070, 12111, 12151, 12191, 12232,
       12272, 12313, 12353, 12393, 12434, 12474, 12515, 12555, 12595,
       12636, 12676, 12717, 12757, 12797, 12838, 12878, 12919, 12959,
       13000])

In [17]:
version

'v015'

In [16]:
counter = 0

for corte in puntos_corte:
    
    envios = np.zeros(len(X_test_with_predictions), dtype=int)
    envios[:corte] = 1
    X_test_with_predictions['Predicted'] = envios
    
    output = X_test_with_predictions[['numero_de_cliente', 'Predicted']]
        
    counter += 1
    
    file_name = f'results_{version}_{counter}.csv'
    output_path = base_path + f'exp/competencia_2/{version}/' + file_name
    
    output.to_csv(output_path, index=False)
            
    print(f'{counter}. corte: {corte}')

1. corte: 9000
2. corte: 9040
3. corte: 9080
4. corte: 9121
5. corte: 9161
6. corte: 9202
7. corte: 9242
8. corte: 9282
9. corte: 9323
10. corte: 9363
11. corte: 9404
12. corte: 9444
13. corte: 9484
14. corte: 9525
15. corte: 9565
16. corte: 9606
17. corte: 9646
18. corte: 9686
19. corte: 9727
20. corte: 9767
21. corte: 9808
22. corte: 9848
23. corte: 9888
24. corte: 9929
25. corte: 9969
26. corte: 10010
27. corte: 10050
28. corte: 10090
29. corte: 10131
30. corte: 10171
31. corte: 10212
32. corte: 10252
33. corte: 10292
34. corte: 10333
35. corte: 10373
36. corte: 10414
37. corte: 10454
38. corte: 10494
39. corte: 10535
40. corte: 10575
41. corte: 10616
42. corte: 10656
43. corte: 10696
44. corte: 10737
45. corte: 10777
46. corte: 10818
47. corte: 10858
48. corte: 10898
49. corte: 10939
50. corte: 10979
51. corte: 11020
52. corte: 11060
53. corte: 11101
54. corte: 11141
55. corte: 11181
56. corte: 11222
57. corte: 11262
58. corte: 11303
59. corte: 11343
60. corte: 11383
61. corte: 114